import os
os.getcwd()
import sys
sys.path.append("/Users/marek/.virtualenvs/bdg-sequila/lib/python3.7/site-packages")
sys.path.append("/Users/marek/git/forks/bdg-sequila/python")

In [10]:
import findspark
from pyspark.sql import SparkSession, DataFrame, SQLContext
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
.builder \
.config('spark.ui.enabled','false') \
.config('spark.driver.memory','2g') \
.config('spark.driver.bindAddress', 'localhost')  \
.config('spark.driver.host', 'localhost') \
.appName('SeQuiLa') \
.config('spark.jars','/Users/marek/git/forks/bdg-sequila/target/scala-2.11/bdg-sequila-assembly-0.5.6-spark-2.4.3.jar')  \
.getOrCreate()

In [ ]:
#config paths
bamPath = '/Users/marek/git/forks/bdg-sequila/src/test/resources/NA12878.slice.bam'
vcfPath = '/Users/marek/git/forks/bdg-sequila/src/test/resources/vcf/test.vcf'

In [12]:
#register sequila.py
from sequila import SequilaSession
ss = SequilaSession(spark)

In [13]:
# create table
tableNameBAM =  'test'
ss.sql(f'CREATE TABLE {tableNameBAM} \
USING org.biodatageeks.datasources.BAM.BAMDataSource \
OPTIONS(path "{bamPath}")')

DataFrame[]

In [14]:
# query table
ss.sql(f"SELECT * FROM {tableNameBAM} LIMIT 1").show()

+--------+----------+-----+---+-----+----+--------------------+--------------------+-----+----------+---------+
|sampleId|contigName|start|end|cigar|mapq|               baseq|            sequence|flags|materefind|SAMRecord|
+--------+----------+-----+---+-----+----+--------------------+--------------------+-----+----------+---------+
| NA12878|      chr1|   34|109|  76M|   2|ACCCADEFFECEFGGFD...|CCCTAACCCTAACCCTA...|   99|         1|     null|
+--------+----------+-----+---+-----+----+--------------------+--------------------+-----+----------+---------+



In [15]:
ss.sql("show tables").show()
ss.sql(f"SELECT * FROM {tableNameBAM} LIMIT 1").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
| default|     test|      false|
+--------+---------+-----------+

+--------+----------+-----+---+-----+----+--------------------+--------------------+-----+----------+---------+
|sampleId|contigName|start|end|cigar|mapq|               baseq|            sequence|flags|materefind|SAMRecord|
+--------+----------+-----+---+-----+----+--------------------+--------------------+-----+----------+---------+
| NA12878|      chr1|   34|109|  76M|   2|ACCCADEFFECEFGGFD...|CCCTAACCCTAACCCTA...|   99|         1|     null|
+--------+----------+-----+---+-----+----+--------------------+--------------------+-----+----------+---------+



In [16]:
ss.sql(f"SELECT * FROM bdg_coverage('{tableNameBAM}','NA12878', 'blocks') LIMIT 5").show()

+----------+-----+---+--------+
|contigName|start|end|coverage|
+----------+-----+---+--------+
|      chr1|   34| 34|       1|
|      chr1|   35| 35|       2|
|      chr1|   36| 37|       3|
|      chr1|   38| 40|       4|
|      chr1|   41| 49|       5|
+----------+-----+---+--------+



In [17]:
tableNameVCF =  'test_vcf'
ss.sql(f'CREATE TABLE {tableNameVCF} \
USING org.biodatageeks.datasources.VCF.VCFDataSource \
OPTIONS(path "{vcfPath}")')

DataFrame[]

In [18]:
ss.sql(f"SELECT * FROM {tableNameVCF} LIMIT 5").show()

+------+-----+-----+-----+---------+---+---+----+------+--------------------+---+---+---+-----+---------+
|contig|  pos|start| stop|       id|ref|alt|qual|filter|                info| gt| gq| dp|   hq|sample_id|
+------+-----+-----+-----+---------+---+---+----+------+--------------------+---+---+---+-----+---------+
|    20|14370|14369|14370|rs6054257|  G|  A|  29|  PASS|[ns -> 3, db -> D...|0|0| 48|  1|51,51|  NA00001|
|    20|14370|14369|14370|rs6054257|  G|  A|  29|  PASS|[ns -> 3, db -> D...|1|0| 48|  8|51,51|  NA00002|
|    20|14370|14369|14370|rs6054257|  G|  A|  29|  PASS|[ns -> 3, db -> D...|1/1| 43|  5|  .,.|  NA00003|
|    20|17330|17329|17330|        .|  T|  A|   3|   q10|[ns -> 3, dp -> 1...|0|0| 49|  3|58,50|  NA00001|
|    20|17330|17329|17330|        .|  T|  A|   3|   q10|[ns -> 3, dp -> 1...|0|1|  3|  5| 65,3|  NA00002|
+------+-----+-----+-----+---------+---+---+----+------+--------------------+---+---+---+-----+---------+



In [8]:
ss.stop()